# Tutorial: Filter Documents Based on Metadata

- **Level**: Beginner
- **Time to complete**: 5 minutes
- **Components Used**: [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/v2.0/docs/inmemorydocumentstore), [`InMemoryBM25Retriever`](https://docs.haystack.deepset.ai/v2.0/docs/inmemorybm25retriever)
- **Prerequisites**: None
- **Goal**: Filter documents in a document store based on given metadata

## Overview

**📚 Useful Documentation: [Metadata Filtering](https://docs.haystack.deepset.ai/v2.0/docs/metadata-filtering)**

Although new retrieval techniques are great, sometimes you just know that you want to perform search on a specific group of documents in your document store. This can be anything from all the documents that are related to a specific _user_, or that were published after a certain _date_ and so on. Metadata filtering is very useful in these situations. In this tutorial, we will a few simple documents containing information about Haystack, where the metadata includes information on what version of Haystack the information relates to. We will then do metadata filtering to make sure we are answering the question based only on information about Haystack 2.0.


## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/log-level)

## Installing Haystack

Install Haystack 2.0 Beta with `pip`:

In [1]:
%%bash

pip install haystack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


### Enabling Telemetry

Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/v2.0/docs/enabling-telemetry) for more details.

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(31)

## Preparing Documents

First, let's prepare some documents. Below, we're manually creating 3 simple documents with `meta` attached. We're then writing these documents to an `InMemoryDocumentStore`, but you can [use any of the available document stores](https://docs.haystack.deepset.ai/v2.0/docs/choosing-a-document-store) instead such as OpenSearch, Chroma, Pinecone and more.. (Note that not all of them have options to store in memory and may require extra setup).

> ⭐️ For more information on how to write documents into different document stores, you can follow our tutorial on indexing different file types.

In [7]:
from datetime import datetime

from haystack import  Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever

documents = [Document(content="Use pip to install a basic version of Haystack's latest release: pip install farm-haystack. All the core Haystack components live in the haystack repo. But there's also the haystack-extras repo which contains components that are not as widely used, and you need to install them separately.",
                      meta={"version": 1.15, "date": datetime(2023, 3, 30)}),
             Document(content="Use pip to install a basic version of Haystack's latest release: pip install farm-haystack[inference]. All the core Haystack components live in the haystack repo. But there's also the haystack-extras repo which contains components that are not as widely used, and you need to install them separately.",
                      meta={"version": 1.22, "date": datetime(2023, 11, 7)}),
             Document(content="Use pip to install only the Haystack 2.0 code: pip install haystack-ai. The haystack-ai package is built on the main branch which is an unstable beta version, but it's useful if you want to try the new features as soon as they are merged.",
                      meta={"version": 2.0, "date": datetime(2023, 12, 4)}),
]
document_store = InMemoryDocumentStore(bm25_algorithm="BM25Plus")
document_store.write_documents(documents=documents)

3

## Building a Document Search Pipeline

As an example, below we are building a simple document search pipeline that simplt has a retriever. However, you can also change this pipeline to do more, such as generating answers to questions or more.

In [8]:
from haystack import Pipeline

pipeline = Pipeline()
pipeline.add_component(instance=InMemoryBM25Retriever(document_store=document_store), name="retriever")

## Do Metadata Filtering

Finally, ask a question by filtering the documents to `"version" > 1.21`.

To see what kind of comparison operators you can use for your metadata, including logical comparistons such as `NOT`, `AND` and so on, check out the [Metadata Filtering documentation](https://docs.haystack.deepset.ai/v2.0/docs/metadata-filtering#comparison)

In [14]:
query = "Haystack installation"
pipeline.run(data={"retriever": {"query": query, "filters": { "field": "meta.version", "operator": ">", "value": 1.21}}})

Ranking by BM25...:   0%|          | 0/2 [00:00<?, ? docs/s]

{'retriever': {'documents': [Document(id=b53625c67fee5ba5ac6dc86e7ca0adff567bf8376e86ae4b3fc6f6f858ccf1e5, content: 'Use pip to install a basic version of Haystack's latest release: pip install farm-haystack[inference...', meta: {'version': 1.22, 'date': datetime.datetime(2023, 11, 7, 0, 0)}, score: 1.1808764808011376),
   Document(id=8ac1f8119bdec5c898d5a5c69f49ff47f64056bce1a0f95073e34493bbaf9354, content: 'Use pip to install only the Haystack 2.0 code: pip install haystack-ai. The haystack-ai package is b...', meta: {'version': 2.0, 'date': datetime.datetime(2023, 12, 4, 0, 0)}, score: 1.0867343954443756)]}}

As a final step, let's see how we can add logical operators to our filters. This time, we are asking for retrieved documents to be filtered to `version > 1.21` _AND_ we're also asking their `date` to be kater than November 7th 2023.

In [16]:
query = "Haystack installation"
pipeline.run(data={"retriever": {"query": query, "filters": { "operator": "AND",
      			                                                           "conditions": [{ "field": "meta.version", "operator": ">", "value": 1.21},
                                                                                      { "field": "meta.date", "operator": ">", "value": datetime(2023, 11, 7)}]}}})

Ranking by BM25...:   0%|          | 0/1 [00:00<?, ? docs/s]

{'retriever': {'documents': [Document(id=8ac1f8119bdec5c898d5a5c69f49ff47f64056bce1a0f95073e34493bbaf9354, content: 'Use pip to install only the Haystack 2.0 code: pip install haystack-ai. The haystack-ai package is b...', meta: {'version': 2.0, 'date': datetime.datetime(2023, 12, 4, 0, 0)}, score: 1.8483924814931876)]}}

🎉 Congratulations! You've filtered retrieved documents with metadata!